In [2]:
import pandas as pd
import numpy as np

df_emission_type = pd.read_csv('co2-fossil-plus-land-use.csv')
df_emission_type = df_emission_type[~((df_emission_type['Code'].isna())|(df_emission_type['Code']=='OWID_WRL'))]
df_emission_type = df_emission_type.rename(columns={
    'Annual CO₂ emissions including land-use change': 'Annual CO2 emissions including land-use change',
    'Annual CO₂ emissions from land-use change': 'Annual CO2 emissions from land-use change',
    'Annual CO₂ emissions': 'Annual CO2 emissions'
    })
df_emission_type = df_emission_type[df_emission_type['Year']==max(df_emission_type['Year'])]
df_emission_type

,Entity,Code,Year,Annual CO2 emissions including land-use change,Annual CO2 emissions from land-use change,Annual CO2 emissions
172,Afghanistan,AFG,2022,1.219331e+07,4.536032e+04,12147948.0
691,Albania,ALB,2022,5.058494e+06,1.037645e+05,4954730.0
864,Algeria,DZA,2022,1.782703e+08,1.925176e+06,176345140.0
1037,Andorra,AND,2022,3.653108e+05,-3.334240e+03,368645.0
1210,Angola,AGO,2022,5.326284e+07,3.719257e+07,16070269.0
...,...,...,...,...,...,...
39873,Vietnam,VNM,2022,5.953420e+08,2.517353e+08,343606620.0
39906,Wallis and Futuna,WLF,2022,NaN,NaN,26461.0
40252,Yemen,YEM,2022,9.319126e+06,-2.037221e+06,11356347.0
40425,Zambia,ZMB,2022,6.999363e+07,6.107174e+07,8921889.0


In [3]:
df_mapping_region = pd.read_csv('country-region-mapping.csv')
df_emission_type = pd.merge(left=df_emission_type, right=df_mapping_region, how='left')
df_emission_type

,Entity,Code,Year,Annual CO2 emissions including land-use change,Annual CO2 emissions from land-use change,Annual CO2 emissions,Region
0,Afghanistan,AFG,2022,1.219331e+07,4.536032e+04,12147948.0,Asia
1,Albania,ALB,2022,5.058494e+06,1.037645e+05,4954730.0,Europe
2,Algeria,DZA,2022,1.782703e+08,1.925176e+06,176345140.0,Africa
3,Andorra,AND,2022,3.653108e+05,-3.334240e+03,368645.0,Europe
4,Angola,AGO,2022,5.326284e+07,3.719257e+07,16070269.0,Africa
...,...,...,...,...,...,...,...
212,Vietnam,VNM,2022,5.953420e+08,2.517353e+08,343606620.0,Asia
213,Wallis and Futuna,WLF,2022,NaN,NaN,26461.0,Oceania
214,Yemen,YEM,2022,9.319126e+06,-2.037221e+06,11356347.0,Asia
215,Zambia,ZMB,2022,6.999363e+07,6.107174e+07,8921889.0,Africa


In [4]:
top_n = 3
df_emission_type['Rank'] = df_emission_type.groupby('Region')['Annual CO2 emissions including land-use change'].rank(method='first', ascending=False)
df_emission_type = df_emission_type[df_emission_type['Rank']<=top_n].reset_index(drop=True)
# df_emission_type['Annual CO2 emissions including land-use change'] = df_emission_type['Annual CO2 emissions including land-use change'].round(-6)
# df_emission_type['Annual CO2 emissions from land-use change'] = df_emission_type['Annual CO2 emissions from land-use change'].round(-5)
# df_emission_type['Annual CO2 emissions'] = df_emission_type['Annual CO2 emissions'].round(-6)
for c in ['Annual CO2 emissions from land-use change', 'Annual CO2 emissions']:
    # df_emission_type[c] /= 1_000_000
    df_emission_type[c] = np.abs(df_emission_type[c])
    # df_emission_type[c] = df_emission_type[c].astype(int)
df_emission_type['Annual CO2 emissions including land-use change'] = df_emission_type['Annual CO2 emissions from land-use change']+df_emission_type['Annual CO2 emissions']
df_emission_type_region = df_emission_type[['Region', 'Annual CO2 emissions including land-use change', 'Annual CO2 emissions from land-use change', 'Annual CO2 emissions']].groupby('Region').sum().reset_index().rename(columns={'Annual CO2 emissions including land-use change': 'Annual region CO2 emission', 'Annual CO2 emissions from land-use change': 'region land', 'Annual CO2 emissions': 'region fossil'})
df_emission_type = pd.merge(left=df_emission_type, right=df_emission_type_region, how='left')
df_emission_type = df_emission_type.sort_values(by=['Annual region CO2 emission', 'Annual CO2 emissions including land-use change'], ascending=[False, False]).reset_index(drop=True)
df_emission_type = df_emission_type.rename(columns={'Annual CO2 emissions': 'fossil', 'Annual CO2 emissions from land-use change': 'land'})
df_emission_type

,Entity,Code,Year,Annual CO2 emissions including land-use change,land,fossil,Region,Rank,Annual region CO2 emission,region land,region fossil
0,China,CHN,2022,1.237828e+10,9.815037e+08,1.139678e+10,Asia,1.0,1.688983e+10,1.934525e+09,1.495530e+10
1,India,IND,2022,2.864052e+09,3.440793e+07,2.829644e+09,Asia,2.0,1.688983e+10,1.934525e+09,1.495530e+10
2,Indonesia,IDN,2022,1.647497e+09,9.186134e+08,7.288833e+08,Asia,3.0,1.688983e+10,1.934525e+09,1.495530e+10
3,United States,USA,2022,5.199053e+09,1.417495e+08,5.057304e+09,North America,1.0,6.727565e+09,6.103459e+08,6.117219e+09
4,Mexico,MEX,2022,8.626858e+08,3.507138e+08,5.119720e+08,North America,2.0,6.727565e+09,6.103459e+08,6.117219e+09
5,Canada,CAN,2022,6.658265e+08,1.178826e+08,5.479439e+08,North America,3.0,6.727565e+09,6.103459e+08,6.117219e+09
6,Russia,RUS,2022,1.965207e+09,3.130295e+08,1.652177e+09,Europe,1.0,2.973485e+09,3.370489e+08,2.636436e+09
7,Germany,DEU,2022,6.736935e+08,8.088792e+06,6.656047e+08,Europe,2.0,2.973485e+09,3.370489e+08,2.636436e+09
8,United Kingdom,GBR,2022,3.345850e+08,1.593060e+07,3.186544e+08,Europe,3.0,2.973485e+09,3.370489e+08,2.636436e+09
9,Brazil,BRA,2022,1.596469e+09,1.112992e+09,4.834773e+08,South America,1.0,1.952328e+09,1.176268e+09,7.760596e+08


In [15]:
nodes = []
nodes_region = []
for idx, row in df_emission_type.iterrows():
    nodes.append(pd.DataFrame({'node': [row["Entity"]]}))
    if idx%top_n==0:
        nodes_region.append(pd.DataFrame({'node': [row["Region"]]}))

nodes = pd.concat(nodes+nodes_region, ignore_index=True)
nodes = nodes.append(pd.DataFrame({'node': ['fossil']}), ignore_index=True).append(pd.DataFrame({'node': ['land']}), ignore_index=True)
nodes.to_csv('nodes.csv')
nodes

C:\Users\andre\AppData\Local\Temp\ipykernel_24232\1120699724.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nodes = nodes.append(pd.DataFrame({'node': ['fossil']}), ignore_index=True).append(pd.DataFrame({'node': ['land']}), ignore_index=True)


,node
0,China
1,India
2,Indonesia
3,United States
4,Mexico
5,Canada
6,Russia
7,Germany
8,United Kingdom
9,Brazil


In [16]:
links = []
links_region = []
for idx, row in df_emission_type.iterrows():
    links.append(pd.DataFrame({'source': [idx], 'target': [24], 'value': row['fossil']}))
    links.append(pd.DataFrame({'source': [idx], 'target': [25], 'value': row['land']}))
    if idx%top_n==0:
        links_region.append(pd.DataFrame({'source': [24], 'target': [idx//top_n+18], 'value': [row['region fossil']]}))
        links_region.append(pd.DataFrame({'source': [25], 'target': [idx//top_n+18], 'value': [row['region land']]}))

links = pd.concat(links+links_region, ignore_index=True)
links.to_csv('links.csv')
links

,source,target,value
0,0,24,1.139678e+10
1,0,25,9.815037e+08
2,1,24,2.829644e+09
3,1,25,3.440793e+07
4,2,24,7.288833e+08
5,2,25,9.186134e+08
6,3,24,5.057304e+09
7,3,25,1.417495e+08
8,4,24,5.119720e+08
9,4,25,3.507138e+08
